<a href="https://colab.research.google.com/github/sugey-gi/edp1/blob/main/Cuerda_onda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parámetros físicos
L = 1.0      # Longitud de la cuerda (m)
k = 0.1      # Amplitud máxima de deflexión (m)
c = 1.0      # Velocidad de propagación (m/s)

# Discretización espacial y temporal
Nx = 100     # Número de puntos espaciales
Nt = 200     # Número de pasos de tiempo
dx = L / (Nx - 1)
dt = 0.5 * dx / c  # Criterio de estabilidad de Courant

# Mallado espacial
x = np.linspace(0, L, Nx)

# Función de deflexión inicial triangular (vectorizada)
def initial_deflection(x, L, k):
    return np.where(x <= L/2, (2 * k * x) / L, (2 * k * (L - x)) / L)

# Inicialización de matrices de estado
u = np.zeros((Nt, Nx))         # Solución completa
u_prev = initial_deflection(x, L, k)  # u(x, 0)
u_curr = np.zeros(Nx)          # u(x, dt)

# Condiciones iniciales
u[0, :] = u_prev

# Primer paso temporal usando Taylor expansion
u_curr[1:-1] = u_prev[1:-1] + 0.5 * (c * dt / dx)**2 * (
    u_prev[2:] - 2 * u_prev[1:-1] + u_prev[:-2]
)
u[1, :] = u_curr

# Bucle en el tiempo para evolución
for n in range(1, Nt - 1):
    u_next = np.zeros(Nx)
    u_next[1:-1] = 2 * u_curr[1:-1] - u_prev[1:-1] + (c * dt / dx)**2 * (
        u_curr[2:] - 2 * u_curr[1:-1] + u_curr[:-2]
    )
    u_next[0] = 0    # Extremo izquierdo fijo
    u_next[-1] = 0   # Extremo derecho fijo

    # Actualizar estados
    u_prev, u_curr = u_curr, u_next
    u[n + 1, :] = u_curr

# Visualización animada (en Jupyter Notebook)
from IPython.display import display, clear_output
import time

fig, ax = plt.subplots()
line, = ax.plot(x, u[0, :])
ax.set_ylim([-1.5 * k, 1.5 * k])
ax.set_xlabel('Posición x')
ax.set_ylabel('Deflexión u(x, t)')

for n in range(0, Nt, 10):
    line.set_ydata(u[n, :])
    ax.set_title(f'Evolución de la Cuerda Vibratoria (Tiempo: {n * dt:.3f} s)')
    display(fig)
    clear_output(wait=True)
    time.sleep(0.01)

plt.close(fig)